In [8]:
import os
os.chdir('C:/Users/v-kirdwivedi/Documents/GitHub/MatchingAlgorithm/')

In [9]:
import pandas as pd
import numpy as np
from funcs import *
from multi_funcs import *
from tarjan_alg import *

### Step-by-Step from Matches and Preferences to Graph

Here I discuss step-by-step how I go from the output of the Gale-Shapley Algorithm to the graph used to identify SCC. 

#### 0: Run GS Algorithm
Before we begin we need to have a preferences data frame and run the gale-shapley algorithm on it. After doing so, we have two dataframes: (1) a preferences dataframe, (2) a matches dataframe. We use both of these in the construction of the graph. 

In [23]:
preferences = mdf_np(10,3) ## Create a preferences dataframe for this example.
preferences_2 = preferences.copy() ## Create a copy
preferences

,0,1,2,student_id,applications,k,matched,rank1,rank2,rank3,underdemanded
0,1,7,2,0,0,0,False,0.830953,0.364250,0.324724,True
1,7,1,4,1,0,0,False,0.752514,0.808902,0.578260,True
2,8,4,6,2,0,0,False,0.028473,0.181069,0.690227,True
3,5,6,4,3,0,0,False,0.518750,0.192062,0.722550,True
4,8,4,1,4,0,0,False,0.345938,0.764293,0.650732,True
5,0,2,5,5,0,0,False,0.938795,0.261503,0.040388,True
6,4,1,8,6,0,0,False,0.724726,0.064546,0.894478,True
7,1,0,6,7,0,0,False,0.869225,0.328311,0.719336,True
8,5,2,7,8,0,0,False,0.483104,0.511075,0.346632,True
9,0,1,3,9,0,0,False,0.509579,0.713523,0.741859,True


In [24]:
matches, _ = run_gale_shapley(preferences_2, k=3) ## Run the Gale-Shapley algorithm on the preferences dataframe.
matches

,0,1,2,student_id,applications,k,matched,rank1,rank2,rank3,underdemanded
0,7,1,4,1,0,1,False,0.752514,0.808902,0.578260,False
1,6,6,6,2,2,1,False,0.690227,0.690227,0.690227,True
2,5,6,4,3,0,1,False,0.518750,0.192062,0.722550,False
3,8,4,1,4,0,1,False,0.345938,0.764293,0.650732,False
4,0,2,5,5,0,1,False,0.938795,0.261503,0.040388,False
5,4,1,8,6,0,1,False,0.724726,0.064546,0.894478,False
6,1,0,6,7,0,1,False,0.869225,0.328311,0.719336,False
7,2,7,7,8,1,1,False,0.511075,0.346632,0.346632,False
8,3,3,3,9,2,1,False,0.741859,0.741859,0.741859,True


Note that the "applications" column here represents the number of times an agent has been rejected. For k=3, this can be at most 2 if they are still matched. Note also not all students are in the matches dataframe. Some are eliminated because they fail to find a match. Note also that the matches are effectively given by the "0" column and the "student_id" column. 

## 1: Drop Irrelevant Students
Students that failed to match and students that matched to their first choice will not be part of SCCs since the former will have no edges directed towards them and the latter will have no edges directed away from them. We drop students in either group from the preferences dataframe: 

In [25]:
preferences = preferences[preferences['student_id'].isin(matches['student_id'])] ## Only keep students who were matched
preferences.reset_index(inplace = True, drop = True) 

In [26]:
preferences['rejections'] = matches.applications
relevant = preferences[preferences['rejections'] != 0] ## drop all who were never rejected. They will necessarily not point to anyone else.

In [29]:
relevant

,0,1,2,student_id,applications,k,matched,rank1,rank2,rank3,underdemanded,rejections
1,8,4,6,2,0,0,False,0.028473,0.181069,0.690227,True,2
7,5,2,7,8,0,0,False,0.483104,0.511075,0.346632,True,1
8,0,1,3,9,0,0,False,0.509579,0.713523,0.741859,True,2


We are now left with only relevant vertices. These are students who have been rejected at least once but that did eventually match. For these people we have to create a graph. 

#### 2: Keep preferences above match
For each of these remaining students we have to create edges that run between them and the agents at schools they would have preferred. For those agents that were rejected once, they'll have one outward edge. For those agents that were rejected twice, we can have two edges (for the k=3 case). 

In [30]:
## Go column by column. If the student was rejected from first choice, that school is relevant. So keep. 
## If student rejected from second too, keep it. If not, set value to -100. We will drop this later. 
## etc... up to k-1 which is most number of schools student could be rejected from.
for i in range(1,3):
        relevant.iloc[:, i] = np.where(relevant['rejections']<i+1, -100, relevant.iloc[:, i])

## Set index and "stack" the dataframe so that we get student|preferred school
relevant.set_index('student_id', inplace = True)
pointing = pd.DataFrame(relevant.iloc[:, :3].stack(level = 0)).reset_index()
pointing = pointing[pointing[0] != -100]

In [31]:
pointing

,student_id,level_1,0
0,2,0,8
1,2,1,4
3,8,0,5
6,9,0,0
7,9,1,1


In this new dataframe, student_id is the vertex, "0" gives the **school** they point to. Each school will be matched to a student, so the next step is to map school numbers in "0" to the matched students:

In [32]:
to_merge = matches.loc[:,[0, 'student_id']]
pointing = pointing.merge(to_merge, on = 0, how = 'left')

In [33]:
pointing

,student_id_x,level_1,0,student_id_y
0,2,0,8,4
1,2,1,4,6
2,8,0,5,3
3,9,0,0,5
4,9,1,1,7


So now we have students pointing to other students. 

#### 3: Run Tarjan's Algorithm

Now that we have a dataframe of directed edges, we can run Tarjan. First, however, we normalize the student_IDs in the above. This is because the code of Tarjan we use relies on vertices being numbered from 0 up to N where N is the size of the graph. We do this by order of appearance going down and then across. So, in the above example we map 2 -> 0, 8->1, 9->2, 4->3, 6->4 etc... This should not change the number of SCCs, since all we are doing is changing the "names" of our vertices. 

In [34]:
normalizer = pd.DataFrame(pd.concat([pointing['student_id_x'], pointing['student_id_y']], axis = 0).unique())
normalizer['new_id'] = normalizer.index
pointing = pointing.merge(normalizer, left_on = 'student_id_x', right_on = 0, how = 'left')
pointing = pointing.merge(normalizer, left_on = 'student_id_y', right_on = 0, how = 'left')
pairs = pointing[['new_id_x', 'new_id_y']]
    

In [35]:
pairs

,new_id_x,new_id_y
0,0,3
1,0,4
2,1,5
3,2,6
4,2,7


In [36]:
g = Graph(len(normalizer)) ## init graph
for i in range(len(pairs)): ## Add edges
    g.addEdge(pairs.iloc[i, 0], pairs.iloc[i, 1])
g.SCC() ## run SCC algorithm

(0, [])

In this case, no SCC. 